In [11]:
!pip install openai==0.28.0
!pip install textstat

In [12]:
import os
import openai
import pandas as pd
import joblib

# Set the API key as an environment variable in the Jupyter Lab
# os.environ['OPENAI_API_KEY'] = 'Enter your API key here'
# openai.api_key = os.getenv('OPENAI_API_KEY')

df = pd.read_csv('data/processed_data.csv')
model = joblib.load('models/churn_model.pkl')

# Sample marketing offers data
marketing_offers = [
    {"customer_profile": "age: 30, plan: basic", "message": "Hi there! Upgrade to our premium plan and enjoy exclusive benefits."},
    {"customer_profile": "age: 45, plan: premium", "message": "Thank you for being a loyal customer! Enjoy a 20% discount on your next bill."},
    {"customer_profile": "age: 25, plan: basic", "message": "Special offer for you! Switch to our premium plan and get the first month free."},
    {"customer_profile": "age: 35, plan: premium", "message": "We appreciate your loyalty. Here’s a 30% discount on additional services."}
]

# Convert the sample marketing offers data to a DataFrame
marketing_df = pd.DataFrame(marketing_offers)

# Function to generate a personalized message using GPT-3
def generate_message(customer_profile, offer):
    profile_str = ", ".join(f"{key}: {value}" for key, value in customer_profile.items())
    prompt = f"Create a personalized marketing message for a customer with the following profile: {profile_str}. Offer: {offer}\n\nExamples:\n"
    
    for _, row in marketing_df.iterrows():
        prompt += f"Profile: {row['customer_profile']}\nMessage: {row['message']}\n\n"
    
    prompt += f"Profile: {profile_str}\nMessage:"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt}]
    )
    
    return response.choices[0].message['content'].strip()


# Function to integrate the churn prediction and message generation
def chatbot_interface(customer_index, df, model):
    features = df.drop(columns=['Churn'])
    
    # Predict churn risk
    customer_data = features.iloc[[customer_index]]  # Ensure customer_data is a DataFrame with correct feature names
    churn_risk = model.predict(customer_data)
    
    # Generate customer profile excluding the target column 'Churn'
    customer_profile = df.drop(columns=['Churn']).iloc[customer_index].to_dict()
    if churn_risk == 1:
        message = generate_message(customer_profile, "20% discount on next bill")
        return {
            'message': message
        }
    else:
        return {
            'message': "Customer is not at high risk of churn.",'clarity': 0,'persuasion': 0,'personalization': 0
        }

In [13]:
import re
import textstat
# Define a function to check for persuasive words
def count_persuasive_words(text):
    persuasive_words = {"discount":1.5, "exclusive":1.4, "special":1.5, "offer":1.8, "free":2,
       "bonus":1.2, "premium":1.2,"reward":1.5, "gratitude":1.2, "appreciate":1.5, "thanks":1.3}
    count = sum(weight for word, weight in persuasive_words.items() if word in text.lower())
    return count, sum(persuasive_words.values())

# Define a function to check for personalization
def check_personalization(message, customer_profile):
    personalized_phrases = {"notice":1.5, "specially":2, "continue":1.2, "value":1.3, "loyalty":1.5}
    personalized_count = sum(weight for phrase, weight in personalized_phrases.items() if phrase in message.lower())
    total_personalized = sum(personalized_phrases.values())
    
    return personalized_count, total_personalized

# Automated evaluation function
def evaluate_message(message, customer_profile):
    # Clarity: Using readability score
    clarity = textstat.flesch_reading_ease(message)
    
    # Persuasion: Count persuasive words and calculate percentage
    persuasive_count,total_persuasive = count_persuasive_words(message)
    persuasion = round(((persuasive_count / total_persuasive) * 100),2)
    
    # Personalization: Check for specific customer details and calculate percentage
    personalized_count, total_personalized = check_personalization(message, customer_profile)
    personalization = round(((personalized_count / total_personalized) * 100),2)
    
    return clarity, persuasion, personalization

In [14]:
def run_chatbot():
    while True:
        customer_index = input("Enter Customer ID (or type 'exit' to quit): ")
        if customer_index.lower() == 'exit':
            break
        
        try:
            customer_index = int(customer_index)
            response = chatbot_interface(customer_index, df, model)
            print(response)
            
            view_parameters = input("Type 'Y' if you want to analyze the response parameters (or type 'N' to quit): ")
            if view_parameters.lower() == 'y':
                message = response['message']
                customer_profile = df.drop(columns=['Churn']).iloc[customer_index].to_dict()
                clarity, persuasion, personalization = evaluate_message(message, customer_profile)
                print({'clarity': clarity, 'persuasion': persuasion, 'personalization': personalization})
            else:
                continue
        except ValueError:
            print("Invalid input. Please enter a numeric Customer ID.")
        except KeyError:
            print("Customer ID not found. Please enter a valid Customer ID.")
        except Exception as e:
            print(f"An error occurred: {e}")

run_chatbot()

Enter Customer ID (or type 'exit' to quit):  68


{'message': "Hi there, thank you for being a valued customer! As a token of appreciation, we're excited to offer you a 20% discount on your next bill. Enjoy the savings and continue to stay connected with us."}


Type 'Y' if you want to analyze the response parameters (or type 'N' to quit):  yes
Enter Customer ID (or type 'exit' to quit):  y


Invalid input. Please enter a numeric Customer ID.


Enter Customer ID (or type 'exit' to quit):  49


{'message': "Hi there! We appreciate your continued loyalty as a valued customer. As a token of our gratitude, we're delighted to offer you a 20% discount on your next bill. Thank you for choosing us as your service provider."}


Type 'Y' if you want to analyze the response parameters (or type 'N' to quit):  Y


{'clarity': 67.04, 'persuasion': 37.27, 'personalization': 53.33}


Enter Customer ID (or type 'exit' to quit):  exit
